# Import fonctions

In [ ]:
import os
import numpy as np
from PIL import Image
from glob import glob
import tensorflow as tf
import mlflow
import mlflow.tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, BatchNormalization
from tensorflow.keras.optimizers import Adam
from concurrent.futures import ThreadPoolExecutor
from functools import partial


def _load_and_process_pair(img_path, mask_path, image_size):
    img = Image.open(img_path).convert("RGB").resize((image_size[1], image_size[0]))
    mask = Image.open(mask_path).convert("L").resize((image_size[1], image_size[0]), resample=Image.NEAREST)
    return np.array(img) / 255.0, np.array(mask)

def load_images_and_masks(image_dir, mask_dir, image_size=(256, 256), num_workers=40):
    image_paths = sorted(glob(os.path.join(image_dir, '*/*.png')))
    mask_paths = sorted(glob(os.path.join(mask_dir, '*/*.png')))

    print(f"📂 {len(image_paths)} images trouvées dans {image_dir}")
    print(f"📂 {len(mask_paths)} masques trouvés dans {mask_dir}")

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = list(executor.map(partial(_load_and_process_pair, image_size=image_size), image_paths, mask_paths))

    images, masks = zip(*results)
    images = np.array(images, dtype=np.float32)
    masks = np.array(masks, dtype=np.uint8)
    masks = np.expand_dims(masks, axis=-1)

    return images, masks


# === UNet modulaire ===
def double_conv(x, filters):
    x = Conv2D(filters, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    return x

def build_unet(input_shape=(256, 256, 3), num_classes=8, depth=2, base_filters=32):
    inputs = Input(shape=input_shape)

    # Encodeur
    encoders = []
    x = inputs
    for i in range(depth):
        filters = base_filters * (2 ** i)
        x = double_conv(x, filters)
        encoders.append(x)
        x = MaxPooling2D()(x)

    # Bottleneck
    x = double_conv(x, base_filters * (2 ** depth))

    # Decodeur
    for i in reversed(range(depth)):
        filters = base_filters * (2 ** i)
        x = UpSampling2D()(x)
        x = concatenate([x, encoders[i]])
        x = double_conv(x, filters)

    outputs = Conv2D(num_classes, 1, activation='softmax')(x)
    return Model(inputs, outputs)


# === Fonctions de loss et metrics ===

def sparse_categorical_crossentropy_ignore_255(y_true, y_pred):
    y_true = tf.squeeze(y_true, axis=-1)
    mask = tf.not_equal(y_true, 255)
    y_true_clean = tf.where(mask, y_true, tf.zeros_like(y_true))
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true_clean, y_pred)
    mask = tf.cast(mask, loss.dtype)
    loss = loss * mask
    return tf.reduce_sum(loss) / (tf.reduce_sum(mask) + 1e-8)


def masked_mean_iou(y_true, y_pred):
    num_classes = 8
    y_true = tf.squeeze(y_true, axis=-1)
    mask = tf.not_equal(y_true, 255)
    y_true = tf.where(mask, y_true, 0)
    y_pred = tf.argmax(y_pred, axis=-1, output_type=tf.int32)

    iou_list = []
    for i in range(num_classes):
        y_true_i = tf.equal(y_true, i)
        y_pred_i = tf.equal(y_pred, i)
        intersection = tf.reduce_sum(tf.cast(y_true_i & y_pred_i, tf.float32))
        union = tf.reduce_sum(tf.cast(y_true_i | y_pred_i, tf.float32))
        iou = tf.math.divide_no_nan(intersection, union)
        iou_list.append(iou)
    return tf.reduce_mean(iou_list)


def masked_dice_coef(y_true, y_pred):
    y_true = tf.squeeze(y_true, axis=-1)
    mask = tf.not_equal(y_true, 255)
    y_true = tf.where(mask, y_true, 0)
    y_pred = tf.argmax(y_pred, axis=-1, output_type=tf.int32)

    y_true_flat = tf.reshape(y_true, [-1])
    y_pred_flat = tf.reshape(y_pred, [-1])
    mask_flat = tf.reshape(mask, [-1])

    y_true_flat = tf.boolean_mask(y_true_flat, mask_flat)
    y_pred_flat = tf.boolean_mask(y_pred_flat, mask_flat)

    dice_list = []
    num_classes = 8
    for i in range(num_classes):
        y_true_c = tf.cast(tf.equal(y_true_flat, i), tf.float32)
        y_pred_c = tf.cast(tf.equal(y_pred_flat, i), tf.float32)
        intersection = tf.reduce_sum(y_true_c * y_pred_c)
        dice = (2. * intersection) / (tf.reduce_sum(y_true_c) + tf.reduce_sum(y_pred_c) + 1e-8)
        dice_list.append(dice)
    return tf.reduce_mean(dice_list)


# === Entraînement ===

def train_unet_cityscapes(
    image_size=(512, 512),         # 👈 taille image ajustée
    batch_size=4,
    epochs=10,
    data_dir="/home/romain/work/projet8/notebooks/data",
    experiment_name="unet-segmentation-8classes",
    model_name="unet_light_cityscapes",
    depth=3,                       # 👈 profondeur configurable
    base_filters=64               # 👈 capacité du modèle
):
    import mlflow.tensorflow
    from mlflow.models.signature import infer_signature

    mlflow.set_tracking_uri("http://127.0.0.1:8080")
    mlflow.set_experiment(experiment_name)
    mlflow.tensorflow.autolog()

    print("📦 Chargement des données d'entraînement...")
    train_images, train_masks = load_images_and_masks(
        os.path.join(data_dir, "leftImg8bit/train"),
        os.path.join(data_dir, "masks_8_true/train"),
        image_size=image_size
    )

    print("📦 Chargement des données de validation...")
    val_images, val_masks = load_images_and_masks(
        os.path.join(data_dir, "leftImg8bit/val"),
        os.path.join(data_dir, "masks_8_true/val"),
        image_size=image_size
    )

    with mlflow.start_run():
        print("🧠 Création du modèle U-Net paramétrable...")
        model = build_unet(
            input_shape=(*image_size, 3),
            num_classes=8,
            depth=depth,
            base_filters=base_filters
        )

        model.compile(
            optimizer=Adam(1e-3),
            loss=sparse_categorical_crossentropy_ignore_255,
            metrics=["accuracy", masked_mean_iou, masked_dice_coef]
        )

        print("🚀 Démarrage de l'entraînement...")
        history = model.fit(
            train_images, train_masks,
            validation_data=(val_images, val_masks),
            epochs=epochs,
            batch_size=batch_size
        )

        print("💾 Sauvegarde du modèle localement au format `.keras`...")
        keras_model_path = f"{model_name}.keras"
        model.save(keras_model_path)
        mlflow.log_artifact(keras_model_path)

        print("📦 Génération d’un input_example et de la signature...")
        input_example = train_images[:1]
        pred_example = model.predict(input_example)
        signature = infer_signature(input_example, pred_example)

        print("📦 Enregistrement du modèle dans MLflow...")
        mlflow.tensorflow.log_model(
            model=model,
            artifact_path="model",
            registered_model_name=model_name,
            input_example=input_example,
            signature=signature
        )

    return model, history


# Vérification état GPU

watch -n 1 nvidia-smi

# 1024 x 2048 - Droplet Digital Ocean

In [ ]:
train_unet_cityscapes(
    image_size=(1024, 2048),
    batch_size=10,
    epochs=10,
    depth=4,
    base_filters=64,
    model_name="unet_1024x2048_d4_f64",
    data_dir="/srv/picture_segmentation/projet8_openclassrooms/notebooks/data",
    experiment_name="exp_unet_cityscapes_1024x2048_true_masks"
)

# Téléchargement du modèle entrainé

In [3]:
import mlflow

local_path = mlflow.artifacts.download_artifacts(
    artifact_uri="models:/unet_1024x2048_d4_f64/1",
    dst_path="downloaded_model_1024x2048"
)

print(f"Modèle téléchargé ici : {local_path}")


Modèle téléchargé ici : /srv/picture_segmentation/projet8_openclassrooms/notebooks/downloaded_model_384x768/
